# Corpus finder tests

### Imports

In [20]:
import os
import pickle as pkl
import sys
from random import random

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader



import argparse
import math
import os
import pickle as pkl
import time
from pathlib import Path

import captum.attr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_influence_functions as ptif
import seaborn as sns
import sklearn
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset

from simplexai.explainers.nearest_neighbours import NearNeighLatent
from simplexai.explainers.representer import Representer
from simplexai.explainers.simplex import Simplex
from simplexai.explainers.contex import Contex
from simplexai.models.image_recognition import MnistClassifier
from simplexai.utils.schedulers import ExponentialScheduler



from simplexai.experiments.time_series_prostate_cancer import (
    TimeSeriesProstateCancerDataset,
    load_time_series_prostate_cancer,
)
from simplexai.explainers.simplex import Simplex
from simplexai.models.recurrent_neural_net import MortalityGRU

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path

In [56]:
def load_mnist(
    batch_size: int, train: bool, subset_size=None, shuffle: bool = True
) -> DataLoader:
    dataset = torchvision.datasets.MNIST(
        "./data/",
        train=train,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    )
    if subset_size:
        dataset = torch.utils.data.Subset(
            dataset, torch.randperm(len(dataset))[:subset_size]
        )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

corpus_size = 10

corpus_loader = load_mnist(corpus_size, train=True)
corpus_examples = enumerate(corpus_loader)
_, (corpus_inputs, corpus_target) = next(corpus_examples)
corpus_inputs = corpus_inputs.detach()
test_examples = enumerate(test_loader)
_, (test_inputs, test_target) = next(test_examples)
test_inputs = test_inputs[test_id : test_id + 1].detach()
test_data = test_inputs

print("This is the test_inputs: ")
print(test_inputs.shape)

print("We wish for it to be in this shape: ")
print(corpus_inputs.shape)

new_tensor = [test_inputs] * corpus_size
new_tensor = torch.cat(new_tensor)
print(len(new_tensor))

print("Here is the new tensor: ")
print(new_tensor.shape)

This is the test_inputs: 
torch.Size([1, 1, 28, 28])
We wish for it to be in this shape: 
torch.Size([10, 1, 28, 28])
10
Here is the new tensor: 
torch.Size([10, 1, 28, 28])


In [59]:
def load_mnist(
    batch_size: int, train: bool, subset_size=None, shuffle: bool = True
) -> DataLoader:
    dataset = torchvision.datasets.MNIST(
        "./data/",
        train=train,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    )
    if subset_size:
        dataset = torch.utils.data.Subset(
            dataset, torch.randperm(len(dataset))[:subset_size]
        )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


# Load data:
corpus_size = 500
batch_size_test = 7
test_id = 3

corpus_loader = load_mnist(corpus_size, train=True)
corpus_examples = enumerate(corpus_loader)
_, (corpus_inputs, corpus_target) = next(corpus_examples)
corpus_inputs = corpus_inputs.detach()

test_loader = load_mnist(batch_size_test, train=False)
test_examples = enumerate(test_loader)
_, (test_inputs, test_target) = next(test_examples)

new_tensor = test_inputs
# print(test_target.shape)
# print(test_target)
# print(test_inputs.shape)
# print(test_inputs)

print(4*"\n")

#changed
test_inputs = test_inputs[test_id:test_id+1].detach()
test_inputs = test_inputs[0]
test_input_label = test_target[test_id:test_id+1]

# print(test_input_label.shape)
# print(test_input_label)
# print(test_inputs.shape)
# print(test_inputs)


# print(5* "\n")

##### COUNTERFACTUAL IDXs
N = corpus_size
counterfacts_onehot = np.zeros((N))
for i in range(N):
    if corpus_target[i] != test_input_label:
        counterfacts_onehot[i] = 1
counterfacts_idx = np.where(counterfacts_onehot==1)[0]
counterfacts = corpus_inputs[counterfacts_idx]
N_cf = int(np.sum(counterfacts_onehot))

if N_cf== 0: 
    print("   WARNING: NO COUNTERFACTUAL FOUND")
    quit()

print("target label (test point label) :  ")
print(test_input_label[0])
print("corpus labels :  ")
print(corpus_target)
print("cf potentials :")
print(counterfacts_onehot) 
print("CF indices")
print(counterfacts_idx)
print("\n \n Now we print the second counterfactual's index \n ")
print(counterfacts_idx[1])
print("CFs labels")
print(corpus_inputs[counterfacts_idx].shape)
print("number cfs")
print(N_cf)
print(3*"\n")

print("test_inputs: of dimension  ")
print(test_inputs.shape)
print("flattened: ")
print(torch.flatten(test_inputs).shape)


##### COUNTERFACTUAL Distances
x_test = torch.flatten(test_inputs)
dist_cf = np.zeros((N_cf))
for i in range(N_cf):
    cf_idx = counterfacts_idx[i]
    x_cf = torch.flatten(corpus_inputs[cf_idx])
    print(corpus_inputs[cf_idx].shape)
    print(torch.flatten(corpus_inputs[cf_idx]).shape)
    dist = np.linalg.norm(x_test - x_cf, 2)
    dist_cf[i] = dist

print(2*"\n")
print("Here are the counterfactual distances to test_point")
print(dist_cf )
print(2*"\n")

  
    
##### CALCULATE MASSIVE ADJACENCY MATRIX
# The matrix is composed of the corpus and the test point.
adj = np.zeros((N+1,N+1))
for i in range(N+1):
    if i<N:
        xi = torch.flatten(corpus_inputs[i])
    else:
        xi = torch.flatten(test_inputs)
        
    for j in range(N+1): 
        if j<N:
            xj = torch.flatten(corpus_inputs[j])
        else:
            xj = torch.flatten(test_inputs)
 
        dist = np.linalg.norm(xi - xj, 2)
        adj[i,j] = dist
        adj[j,i] = dist
 
print("Here is the adjacency matrix")
print(adj.shape)
print(adj)

# Get rid of the 40% longest edges
distances = np.zeros((N+1))
k = int(np.floor(0.2*N))
new_adj = np.zeros((N+1, N+1))

for i in range(N+1): 
    distances = adj[i]
    
    # find k smallest elements of adj[i,:] 
    idx = np.argpartition(distances, k)
    
    # find largest values of those k elements
    # max_dist_k = np.amax(distances[idx])
    max_dist_k = np.percentile(distances, 70)
    
    # discard distances larger than max_dist_k
    distances[distances>=max_dist_k] = 0

    # update the new adjacency matrix 
    new_adj[i] = distances
    
print("\n We have a new adjacency matrix: ")
print(new_adj)



##### CALCULATE SHORTEST PATHS
csgraph = csr_matrix(new_adj)
shortest_paths, predecessors = shortest_path(csgraph, method='auto', directed=True, return_predecessors=True, unweighted=False, overwrite=False, indices=None)

print("\n Shortest paths matrix: ")
print(shortest_paths)

print("\n Predecessors matrix: ")
print(predecessors)



# Find counterfactual with shortest path 
counterfacts_distances = shortest_paths[N][counterfacts_idx]
shortest_cft_dist = np.amin(counterfacts_distances)
shortest_cft_id = np.where(new_adj[N]==shortest_cft_dist)[0]


print("\n Shortest distances between counterfactuals and test point:")
print(counterfacts_distances)
print("Reminder: counterfactual indices are: ")
print(counterfacts_idx)
print("Reminder: counterfactual labels are: ")
print(corpus_target[counterfacts_idx])
print("Reminder: target label is : ")
print(test_input_label[0])

print("\n \n The counterfactual_id is :: ")
print(shortest_cft_id)


# Extract path from test point to counterfactual  
current_idx = shortest_cft_id
path_indices = np.zeros((1))
path_indices[0] = shortest_cft_id
while predecessors[N][current_idx] >= 0:
    current_idx = predecessors[N][current_idx]
    np.append(path_indices, current_idx)
print("\n \n This is the path of counterfactuals: ")
print(path_indices)
    
    

# Extract intermediary example from this path 
if len(path_indices) > 1: 
    if len(path_indices) == 2:
        intermediary_id = path_indices[1]
    else:
        intermediary_id = path_indices[len(path_indices//2)]

# PERFORM SIMPLEX ANALYSIS ON INTERMEDIARY AND COUNTERFACTUAL  
        







target label (test point label) :  
tensor(3)
corpus labels :  
tensor([8, 5, 0, 3, 1, 1, 7, 1, 6, 8, 0, 1, 4, 2, 5, 7, 9, 5, 7, 7, 9, 0, 0, 1,
        1, 5, 5, 9, 3, 6, 0, 1, 4, 1, 0, 1, 1, 8, 8, 1, 1, 8, 2, 6, 0, 0, 2, 2,
        2, 9, 3, 6, 1, 6, 5, 9, 2, 8, 0, 3, 9, 9, 8, 6, 0, 8, 4, 8, 4, 0, 3, 1,
        1, 3, 5, 7, 8, 9, 1, 0, 1, 7, 7, 9, 0, 2, 9, 0, 2, 0, 7, 2, 1, 3, 6, 8,
        4, 6, 3, 7, 8, 3, 3, 7, 9, 5, 4, 9, 7, 1, 5, 8, 7, 0, 5, 3, 6, 0, 0, 2,
        6, 5, 1, 8, 5, 0, 3, 7, 1, 1, 5, 7, 9, 5, 0, 2, 5, 8, 4, 7, 7, 7, 5, 1,
        9, 1, 0, 4, 7, 3, 1, 8, 3, 6, 8, 8, 7, 4, 4, 9, 6, 9, 2, 3, 3, 2, 3, 2,
        8, 2, 1, 8, 6, 9, 3, 7, 1, 9, 6, 8, 9, 8, 3, 9, 0, 9, 1, 1, 8, 0, 0, 5,
        9, 1, 2, 9, 0, 6, 7, 5, 0, 7, 3, 1, 2, 7, 9, 6, 5, 3, 2, 9, 3, 8, 5, 8,
        4, 4, 4, 9, 9, 6, 9, 0, 0, 5, 0, 9, 5, 3, 0, 1, 0, 4, 4, 4, 1, 4, 7, 9,
        7, 2, 6, 2, 2, 7, 9, 3, 7, 0, 0, 1, 1, 4, 8, 3, 7, 7, 0, 7, 1, 9, 0, 4,
        5, 3, 8, 9, 2, 5, 9, 4, 9, 1, 2, 5, 0, 0, 3

Here is the adjacency matrix
(501, 501)
[[ 0.         24.08324432 36.38453674 ... 35.33808899 31.45649529
  35.23031235]
 [24.08324432  0.         31.39158249 ... 34.378685   35.46274185
  29.53841972]
 [36.38453674 31.39158249  0.         ... 34.94286346 40.64636993
  35.59017181]
 ...
 [35.33808899 34.378685   34.94286346 ...  0.         34.2791214
  41.44871521]
 [31.45649529 35.46274185 40.64636993 ... 34.2791214   0.
  41.36396027]
 [35.23031235 29.53841972 35.59017181 ... 41.44871521 41.36396027
   0.        ]]

 We have a new adjacency matrix: 
[[ 0.         24.08324432  0.         ... 35.33808899 31.45649529
  35.23031235]
 [24.08324432  0.         31.39158249 ...  0.          0.
  29.53841972]
 [36.38453674 31.39158249  0.         ... 34.94286346  0.
  35.59017181]
 ...
 [ 0.         34.378685    0.         ...  0.         34.2791214
   0.        ]
 [31.45649529 35.46274185  0.         ... 34.2791214   0.
   0.        ]
 [35.23031235 29.53841972 35.59017181 ...  0.          0.

In [ ]:
# get the data
# within mnist find how they transfer it to Simplex
# manipulate the corpus

def load_mnist(
    batch_size: int, train: bool, subset_size=None, shuffle: bool = True
) -> DataLoader:
    dataset = torchvision.datasets.MNIST(
        "./data/",
        train=train,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    )
    if subset_size:
        dataset = torch.utils.data.Subset(
            dataset, torch.randperm(len(dataset))[:subset_size]
        )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


# Load data:
corpus_size = 20
test_size = 5
device=torch.device

corpus_loader = load_mnist(corpus_size, train=True)
corpus_examples = enumerate(corpus_loader)
batch_id_corpus, (corpus_data, corpus_target) = next(corpus_examples)
corpus_data = corpus_data.detach()

print("\n corpus downloaded: \n")
print(corpus_data.shape)
print("corpus first elem: ")
print(corpus_data[0].shape)
print("\n corpus target \n")
print(corpus_target)

##### CALCULATE ADJACENCY MATRIX
N = corpus_size
print("the corpus size")
print(N)
print(corpus_data.shape[0])
adj = np.zeros((N,N))
avg_distances = np.zeros((N))
for i in range(N):
    avg_dist_i = 0
    for j in range(0, N):        
        xi = torch.flatten(corpus_data[i])
        xj = torch.flatten(corpus_data[j])
        dist = np.linalg.norm(xi - xj, 2)
        adj[i, j] = dist
        avg_dist_i = avg_dist_i + dist
    avg_distances[i] = avg_dist_i / (N-1)

print("\n Here is the adjacency list \n")
print(adj.shape)
print(adj)
print("\n Avg distances \n")
print(avg_distances)
    
    
##### CALCULATE AVG DIST to K nearest neighbour
k = N // 6
for i in range(N):
    avg_dist_i = 0
    A = adj[i]
    idx = np.argpartition(A, k)
    avg_dist_i = np.mean(A[idx[:k]])
    avg_distances[i] = avg_dist_i 

print("\n Avg dist from 2 nearest neighbours \n")
print(avg_distances)

##### KEEP CLOSEST 80th PERCENTILE for new corpus
N_keep = int(np.ceil(0.6*N))
idx = np.argpartition(avg_distances, N_keep)

new_corpus_data = corpus_data[idx[:N_keep]]
print("\n OG Corpus \n")
print(corpus_data.shape)
print("\n new corpus Corpus \n")
print(new_corpus_data.shape)


for idx, elem in enumerate(corpus_data):
    
    if idx==3:
        print("\n \n we are taking the fourth corpus elem \n ")
        print(elem.shape)
        print(elem[0].shape)
        print(elem[0][0].shape)
        print(elem[0][0][0].shape)
        print("\n       new corpus elem \n ")
        new_elem = torch.flatten(elem)
        print(new_elem.shape)
        print(new_elem)
        
        
## Construct the Corpus: 
# construct the corpus
# construct the #size of the corpus


        
"""
print("\n Changing corpus to numpy")
X = corpus_examples.data.numpy()
print(X)
N = len(X)
mask = np.ones(N, dtype=bool)
# kernel density estimator
kde = KernelDensity(kernel='gaussian', bandwidth=3).fit(X)
log_density = kde.score_samples(X)
uniform_log_density = np.log(1/N)
for i in range(0,N):
    if log_density[i] > uniform_log_density:
        # add xi to corpus
        mask[i] = False 
X = X[mask]

self.corpus_examples =  torch.from_numpy(X)
print("\n Corpus selected and updated")

"""

  # def construct_corpus(
    #     self,
    #     eps_dist: float = 1.0,
    # ) -> None: 
    #     """
    #     Constructs and modifies corpus, 
    #     :param eps_dist: 
    #     """
    #     print("\n We are now selectin corpus")
    #     X = self.corpus_examples.data.numpy()
    #     N = len(X)
    #     mask = np.ones(N, dtype=bool)
    #     # kernel density estimator
    #     kde = KernelDensity(kernel='gaussian', bandwidth=3).fit(X)
    #     log_density = kde.score_samples(X)
    #     uniform_log_density = np.log(1/N)
    #     for i in range(0,N):
    #         if log_density[i] > uniform_log_density:
    #             # add xi to corpus
    #             mask[i] = False 
    #     X = X[mask]

        # self.corpus_examples =  torch.from_numpy(X)
        # print("\n Corpus selected and updated")


"""    
    
##### CALCULATE AVG DIST to K nearest neighbour
k = N // 6
for i in range(N):
    avg_dist_i = 0
    A = adj[i]
    idx = np.argpartition(A, k)
    avg_dist_i = np.mean(A[idx[:k]])
    avg_distances[i] = avg_dist_i 

print("\n Avg dist from 2 nearest neighbours \n")
print(avg_distances)

##### KEEP CLOSEST 80th PERCENTILE for new corpus
N_keep = int(np.ceil(0.6*N))
idx = np.argpartition(avg_distances, N_keep)

new_corpus_data = corpus_data[idx[:N_keep]]
print("\n OG Corpus \n")
print(corpus_data.shape)
print("\n new corpus Corpus \n")
print(new_corpus_data.shape)

"""

In [ ]:


"""
batch_id_test, (test_data, test_targets) = next(test_examples)
batch_id_corpus, (corpus_data, corpus_target) = next(corpus_examples)
corpus_data = corpus_data.to(device).detach()
test_data = test_data.to(device).detach()
corpus_latent_reps = classifier.latent_representation(corpus_data).detach()
corpus_probas = classifier.probabilities(corpus_data).detach()
corpus_true_classes = torch.zeros(corpus_probas.shape, device=device)
corpus_true_classes[torch.arange(corpus_size), corpus_target] = 1
test_latent_reps = classifier.latent_representation(test_data).detach()


def fit_explainers(
    device: torch.device,
    explainers_name: list,
    save_path: Path,
    corpus_size: int = 1000,
    test_size: int = 100,
    n_epoch: int = 10000,
    random_seed: int = 42,
    n_keep: int = 5,
    reg_factor_init: float = 0.1,
    reg_factor_final: float = 100,
    cv: int = 0,
    train_only: bool = False,
) -> list:
    torch.random.manual_seed(random_seed + cv)
    explainers = []

    # Load model:
    classifier = MnistClassifier()
    classifier.load_state_dict(torch.load(save_path / f"model_cv{cv}.pth"))
    classifier.to(device)
    classifier.eval()

    

    # Fit ContEx:
    reg_factor_scheduler = ExponentialScheduler(
        reg_factor_init, reg_factor_final, n_epoch
    )
    contex = Contex(
        corpus_examples=corpus_data, corpus_latent_reps=corpus_latent_reps
    )
    contex.construct_corpus(
        eps_dist=1.0,
    )
    contex.fit(
        test_examples=test_data,
        test_latent_reps=test_latent_reps,
        n_epoch=n_epoch,
        reg_factor=reg_factor_init,
        n_keep=n_keep,
        reg_factor_scheduler=reg_factor_scheduler,
    )
    explainers.append(contex)

    # Fit SimplEx:
    reg_factor_scheduler = ExponentialScheduler(
        reg_factor_init, reg_factor_final, n_epoch
    )
    simplex = Simplex(
        corpus_examples=corpus_data, corpus_latent_reps=corpus_latent_reps
    )
    simplex.fit(
        test_examples=test_data,
        test_latent_reps=test_latent_reps,
        n_epoch=n_epoch,
        reg_factor=reg_factor_init,
        n_keep=n_keep,
        reg_factor_scheduler=reg_factor_scheduler,
    )
    explainers.append(simplex)

    """
    # Fit nearest neighbors:
    nn_uniform = NearNeighLatent(
        corpus_examples=corpus_data, corpus_latent_reps=corpus_latent_reps
    )
    nn_uniform.fit(
        test_examples=test_data, test_latent_reps=test_latent_reps, n_keep=n_keep
    )
    explainers.append(nn_uniform)
    nn_dist = NearNeighLatent(
        corpus_examples=corpus_data,
        corpus_latent_reps=corpus_latent_reps,
        weights_type="distance",
    )
    nn_dist.fit(
        test_examples=test_data, test_latent_reps=test_latent_reps, n_keep=n_keep
    )
    explainers.append(nn_dist)
    """

    # Save explainers and data:
    for explainer, explainer_name in zip(explainers, explainers_name):
        explainer_path = save_path / f"{explainer_name}_cv{cv}_n{n_keep}.pkl"
        with open(explainer_path, "wb") as f:
            print(f"Saving {explainer_name} decomposition in {explainer_path}.")
            pkl.dump(explainer, f)
    corpus_data_path = save_path / f"corpus_data_cv{cv}.pkl"
    with open(corpus_data_path, "wb") as f:
        print(f"Saving corpus data in {corpus_data_path}.")
        pkl.dump([corpus_latent_reps, corpus_probas, corpus_true_classes], f)
    test_data_path = save_path / f"test_data_cv{cv}.pkl"
    with open(test_data_path, "wb") as f:
        print(f"Saving test data in {test_data_path}.")
        pkl.dump([test_latent_reps, test_targets], f)
    return explainers




def approximation_quality(
    n_keep_list: list,
    cv: int = 0,
    random_seed: int = 42,
    model_reg_factor=0.1,
    save_path: str = "results/mnist/quality/",
    train_only=False,
) -> None:
    print(
        100 * "-"
        + "\n"
        + "Welcome in the approximation quality experiment for MNIST. \n"
        f"Settings: random_seed = {random_seed} ; cv = {cv}.\n" + 100 * "-"
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #explainers_name = ["simplex", "nn_uniform", "nn_dist", "representer"]
    explainers_name = ["contex", "simplex"]
    print(explainers_name)
    current_path = Path.cwd()
    save_path = current_path / save_path
    # Create saving directory if inexistent
    if not save_path.exists():
        print(f"Creating the saving directory {save_path}")
        os.makedirs(save_path)

    # Training a model, save it
    print(100 * "-" + "\n" + "Now fitting the model. \n" + 100 * "-")
    train_model(
        device=device,
        random_seed=random_seed,
        cv=cv,
        save_path=save_path,
        model_reg_factor=model_reg_factor,
    )

    # Load the model
    classifier = MnistClassifier()
    classifier.load_state_dict(torch.load(save_path / f"model_cv{cv}.pth"))
    classifier.to(device)
    classifier.eval()

    # Fit the explainers
    print(100 * "-" + "\n" + "Now fitting the explainers. \n" + 100 * "-")
    for i, n_keep in enumerate(n_keep_list):
        print(30 * "-" + f"n_keep = {n_keep}" + 30 * "-")
        explainers = fit_explainers(
            device=device,
            random_seed=random_seed,
            cv=cv,
            test_size=100,
            corpus_size=1000,
            n_keep=n_keep,
            save_path=save_path,
            explainers_name=explainers_name,
            train_only=train_only,
        )
        # Print the partial results
        print(100 * "-" + "\n" + "Results. \n" + 100 * "-")
        #for explainer, explainer_name in zip(explainers, explainers_name[:-1]):
        for explainer, explainer_name in zip(explainers, explainers_name):
            latent_rep_approx = explainer.latent_approx()
            latent_rep_true = explainer.test_latent_reps
            output_approx = classifier.latent_to_presoftmax(latent_rep_approx).detach()
            output_true = classifier.latent_to_presoftmax(latent_rep_true).detach()
            latent_r2_score = sklearn.metrics.r2_score(
                latent_rep_true.cpu().numpy(), latent_rep_approx.cpu().numpy()
            )
            output_r2_score = sklearn.metrics.r2_score(
                output_true.cpu().numpy(), output_approx.cpu().numpy()
            )
            print(
                f"{explainer_name} latent r2: {latent_r2_score:.2g} ; output r2 = {output_r2_score:.2g}."
            )

    """
    # Fit the representer explainer (this is only makes sense by using the whole corpus)
    representer = fit_representer(model_reg_factor, save_path, cv)
    latent_rep_true = representer.test_latent_reps
    output_true = classifier.latent_to_presoftmax(latent_rep_true).detach()
    output_approx = representer.output_approx()
    output_r2_score = sklearn.metrics.r2_score(
        output_true.cpu().numpy(), output_approx.cpu().numpy()
    )
    print(f"representer output r2 = {output_r2_score:.2g}.")
    """

"""

def construct_corpus(
        self,
        eps_dist: float = 1.0,
    ) -> None: 
        """
        Constructs and modifies corpus, 
        :param eps_dist: 
        """
        print("\n We are now selectin corpus")
        X = self.corpus_examples.data.numpy()
        N = len(X)
        mask = np.ones(N, dtype=bool)
        # kernel density estimator
        kde = KernelDensity(kernel='gaussian', bandwidth=3).fit(X)
        log_density = kde.score_samples(X)
        uniform_log_density = np.log(1/N)
        for i in range(0,N):
            if log_density[i] > uniform_log_density:
                # add xi to corpus
                mask[i] = False 
        X = X[mask]

        self.corpus_examples =  torch.from_numpy(X)
        print("\n Corpus selected and updated")

### Load the prostate cancer time series data from file

In [ ]:
def load_data(random_seed=42, corpus_size=100, batch_size=50):
    # LOAD DATA from file
    (
        X,
        y,
        feature_names,
        max_time_points,
        rescale_dict,
    ) = load_time_series_prostate_cancer()

    # Get data into shape and produce corpus
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=random_seed, stratify=y
    )

    print(f"Train set contains {len(y_train)} records")
    print(f"Test set contains {len(y_test)} records")
    print(f"{sum(y_train == 1)} training records with a label of 1")
    print(f"{sum(y_train == 0)} training records with a label of 0")
    print(f"{sum(y_test == 1)} test records with a label of 1")
    print(f"{sum(y_test == 0)} test records with a label of 0")

    class_imbalance_weighting = sum(y_train == 0) / len(y_train)

    train_data = TimeSeriesProstateCancerDataset(X_train, y_train)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    test_data = TimeSeriesProstateCancerDataset(X_test, y_test)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    test_examples = enumerate(test_loader)
    batch_id_test, (test_inputs, test_targets) = next(test_examples)

    corpus_loader = DataLoader(train_data, batch_size=corpus_size, shuffle=False)
    corpus_examples = enumerate(corpus_loader)
    batch_id_corpus, (corpus_inputs, corpus_targets) = next(corpus_examples)

    input_baseline = torch.mean(torch.mean(corpus_inputs, 1), 0).expand(
        100, max_time_points, -1
    )  # Baseline tensor of the same shape as corpus_inputs

    return (
        train_loader,
        test_loader,
        corpus_inputs,
        corpus_targets,
        test_inputs,
        test_targets,
        max_time_points,
        feature_names,
        class_imbalance_weighting,
        input_baseline,
        rescale_dict,
    )


# LOAD data
batch_size = 50
corpus_size = 100

(
    train_loader,
    test_loader,
    corpus_inputs,
    corpus_targets,
    test_inputs,
    test_targets,
    max_time_points,
    feature_names,
    class_imbalance_weighting,
    input_baseline,
    rescale_dict,
) = load_data(random_seed=5, corpus_size=corpus_size, batch_size=batch_size)

In [ ]:
# get the data
# within mnist find how they transfer it to Simplex
# manipulate the corpus


def load_mnist(
    batch_size: int, train: bool, subset_size=None, shuffle: bool = True
) -> DataLoader:
    dataset = torchvision.datasets.MNIST(
        "./data/",
        train=train,
        download=True,
        transform=torchvision.transforms.Compose(
            [
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,)),
            ]
        ),
    )
    if subset_size:
        dataset = torch.utils.data.Subset(
            dataset, torch.randperm(len(dataset))[:subset_size]
        )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)



def fit_explainers(
    device: torch.device,
    explainers_name: list,
    save_path: Path,
    corpus_size: int = 1000,
    test_size: int = 100,
    n_epoch: int = 10000,
    random_seed: int = 42,
    n_keep: int = 5,
    reg_factor_init: float = 0.1,
    reg_factor_final: float = 100,
    cv: int = 0,
    train_only: bool = False,
) -> list:
    torch.random.manual_seed(random_seed + cv)
    explainers = []

    # Load model:
    classifier = MnistClassifier()
    classifier.load_state_dict(torch.load(save_path / f"model_cv{cv}.pth"))
    classifier.to(device)
    classifier.eval()

    # Load data:
    corpus_loader = load_mnist(corpus_size, train=True)
    test_loader = load_mnist(test_size, train=train_only)
    corpus_examples = enumerate(corpus_loader)
    test_examples = enumerate(test_loader)
    batch_id_test, (test_data, test_targets) = next(test_examples)
    batch_id_corpus, (corpus_data, corpus_target) = next(corpus_examples)
    corpus_data = corpus_data.to(device).detach()
    test_data = test_data.to(device).detach()
    corpus_latent_reps = classifier.latent_representation(corpus_data).detach()
    corpus_probas = classifier.probabilities(corpus_data).detach()
    corpus_true_classes = torch.zeros(corpus_probas.shape, device=device)
    corpus_true_classes[torch.arange(corpus_size), corpus_target] = 1
    test_latent_reps = classifier.latent_representation(test_data).detach()

    # Fit ContEx:
    reg_factor_scheduler = ExponentialScheduler(
        reg_factor_init, reg_factor_final, n_epoch
    )
    contex = Contex(
        corpus_examples=corpus_data, corpus_latent_reps=corpus_latent_reps
    )
    contex.construct_corpus(
        eps_dist=1.0,
    )
    contex.fit(
        test_examples=test_data,
        test_latent_reps=test_latent_reps,
        n_epoch=n_epoch,
        reg_factor=reg_factor_init,
        n_keep=n_keep,
        reg_factor_scheduler=reg_factor_scheduler,
    )
    explainers.append(contex)

    # Fit SimplEx:
    reg_factor_scheduler = ExponentialScheduler(
        reg_factor_init, reg_factor_final, n_epoch
    )
    simplex = Simplex(
        corpus_examples=corpus_data, corpus_latent_reps=corpus_latent_reps
    )
    simplex.fit(
        test_examples=test_data,
        test_latent_reps=test_latent_reps,
        n_epoch=n_epoch,
        reg_factor=reg_factor_init,
        n_keep=n_keep,
        reg_factor_scheduler=reg_factor_scheduler,
    )
    explainers.append(simplex)

    """
    # Fit nearest neighbors:
    nn_uniform = NearNeighLatent(
        corpus_examples=corpus_data, corpus_latent_reps=corpus_latent_reps
    )
    nn_uniform.fit(
        test_examples=test_data, test_latent_reps=test_latent_reps, n_keep=n_keep
    )
    explainers.append(nn_uniform)
    nn_dist = NearNeighLatent(
        corpus_examples=corpus_data,
        corpus_latent_reps=corpus_latent_reps,
        weights_type="distance",
    )
    nn_dist.fit(
        test_examples=test_data, test_latent_reps=test_latent_reps, n_keep=n_keep
    )
    explainers.append(nn_dist)
    """

    # Save explainers and data:
    for explainer, explainer_name in zip(explainers, explainers_name):
        explainer_path = save_path / f"{explainer_name}_cv{cv}_n{n_keep}.pkl"
        with open(explainer_path, "wb") as f:
            print(f"Saving {explainer_name} decomposition in {explainer_path}.")
            pkl.dump(explainer, f)
    corpus_data_path = save_path / f"corpus_data_cv{cv}.pkl"
    with open(corpus_data_path, "wb") as f:
        print(f"Saving corpus data in {corpus_data_path}.")
        pkl.dump([corpus_latent_reps, corpus_probas, corpus_true_classes], f)
    test_data_path = save_path / f"test_data_cv{cv}.pkl"
    with open(test_data_path, "wb") as f:
        print(f"Saving test data in {test_data_path}.")
        pkl.dump([test_latent_reps, test_targets], f)
    return explainers




def construct_corpus(
        self,
        eps_dist: float = 1.0,
    ) -> None: 
        """
        Constructs and modifies corpus, 
        :param eps_dist: 
        """
        print("\n We are now selectin corpus")
        X = self.corpus_examples.data.numpy()
        N = len(X)
        mask = np.ones(N, dtype=bool)
        # kernel density estimator
        kde = KernelDensity(kernel='gaussian', bandwidth=3).fit(X)
        log_density = kde.score_samples(X)
        uniform_log_density = np.log(1/N)
        for i in range(0,N):
            if log_density[i] > uniform_log_density:
                # add xi to corpus
                mask[i] = False 
        X = X[mask]

        self.corpus_examples =  torch.from_numpy(X)
        print("\n Corpus selected and updated")


### Get trained mortality model

Use the boolean variable `train_model` to either train a new model or load a pre-trained model from file.

In [ ]:
def load_trained_model(model, trained_model_state_path):
    model.load_state_dict(torch.load(trained_model_state_path))
    # model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    model.eval()
    return model


# Get a trained model
save_path = os.path.abspath(
    f"../demonstrator/resources/trained_models/RNN/Time series Prostate Cancer/"
)
cv = 1
# Train model if required
train_model = False

if train_model:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Define parameters
    n_epoch_model = 600
    log_interval = 3
    weight_decay = 1e-6

    # Create the model
    classifier = MortalityGRU(
        input_dim=len(feature_names),
        hidden_dim=5,
        output_dim=1,
        n_layers=1,
    )
    print(classifier)
    classifier.to(device)
    class_weights = torch.FloatTensor([class_imbalance_weighting]).to(device)
    criterion = nn.BCELoss(weight=class_weights)
    optimizer = optim.Adam(classifier.parameters(), weight_decay=weight_decay)

    # Train the model
    print(100 * "-" + "\n" + "Now fitting the model. \n" + 100 * "-")
    train_losses = []
    train_counter = []
    test_losses = []

    def train(epoch):
        correct = 0
        correct_0 = 0
        correct_1 = 0
        num_targets_0 = 0
        num_targets_1 = 0

        classifier.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(device)
            output = classifier(data)
            target = target.type(torch.LongTensor).unsqueeze(1).to(device)
            optimizer.zero_grad()
            loss = criterion(output, target.float())
            loss.backward()
            optimizer.step()

            train_roc_score = roc_auc_score(
                target.cpu().detach(), output.cpu().detach()
            )
            pred = output.round()
            num_targets_0 += len((target[target == 0]))
            num_targets_1 += len((target[target == 1]))
            correct_0 += float((pred[target == 0] == target[target == 0]).sum())
            correct_1 += float((pred[target == 1] == target[target == 1]).sum())
            correct += float((pred == target).sum())
            if batch_idx % log_interval == 0:
                train_losses.append(loss.item())
                train_counter.append(
                    (batch_idx * 128) + ((epoch - 1) * len(train_loader.dataset))
                )

        print(
            "TRAINING:\n"
            f"correct: {correct}/{len(train_loader.dataset)} ({100. * correct / len(train_loader.dataset):.0f}%)\n"
            f"correct 0s: {correct_0}/{num_targets_0} ({100. * correct_0 / num_targets_0:.0f}%)\n"
            f"correct 1s: {correct_1}/{num_targets_1} ({100. * correct_1 / num_targets_1:.0f}%)\n"
            f"Training set: Avg. loss: {loss:.4f}, ROC AUC score: {train_roc_score:.4f}"
        )

    def test():
        classifier.eval()
        test_loss = 0
        correct = 0
        correct_0 = 0
        correct_1 = 0
        num_targets_0 = 0
        num_targets_1 = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                data = data.to(device)
                output = classifier(data)
                target = target.type(torch.LongTensor).unsqueeze(1).to(device)
                test_loss += criterion(output, target.float()).item()
                pred = output.round()

                test_roc_score = roc_auc_score(target.cpu(), output.cpu())
                num_targets_0 += len((target[target == 0]))
                num_targets_1 += len((target[target == 1]))
                correct_0 += float((pred[target == 0] == target[target == 0]).sum())
                correct_1 += float((pred[target == 1] == target[target == 1]).sum())
                correct += float((pred == target).sum())

        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        print(
            "TESTING:\n"
            f"correct: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f})%\n"
            f"correct 0s: {correct_0}/{num_targets_0} ({100. * correct_0 / num_targets_0:.0f})%\n"
            f"correct 1s: {correct_1}/{num_targets_1} ({100. * correct_1 / num_targets_1:.0f})%\n"
            f"Test set:     Avg. loss: {test_loss:.4f}, ROC AUC score: {test_roc_score:.4f}"
        )

    print("Pre-training test.")
    test()
    print("\n\n")
    for epoch in range(1, n_epoch_model + 1):
        print(f"\nepoch: {epoch}")
        train(epoch)
        test()
        torch.save(
            classifier.state_dict(),
            os.path.join(save_path, f"model_cv{cv}.pth"),
        )
        torch.save(
            optimizer.state_dict(),
            os.path.join(save_path, f"optimizer_cv{cv}.pth"),
        )

# Get a trained model
model = MortalityGRU(
    input_dim=len(feature_names),
    hidden_dim=5,
    output_dim=1,
    n_layers=1,
)  # Model should have the BlackBox interface
TRAINED_MODEL_STATE_PATH = os.path.join(save_path, f"model_cv{cv}.pth")
load_trained_model(model, TRAINED_MODEL_STATE_PATH)

# Compute corpus and test model predictions
corpus_predictions = model.forward(corpus_inputs).detach().round()
test_predictions = model.forward(test_inputs).detach().round()

### Define function for sorting examples to match the sorted output from jacobian decomposition

In [ ]:
# sort order function for decomposition
def apply_sort_order(in_list, sort_order):
    if isinstance(in_list, list):
        return [in_list[idx] for idx in sort_order]
    if torch.is_tensor(in_list):
        return [in_list.numpy()[idx] for idx in sort_order]

### Fit SimplEx

In [ ]:
# Fit SimplEx
# Compute the corpus and test latent representations
corpus_latents = model.latent_representation(corpus_inputs).detach()
test_latents = model.latent_representation(test_inputs).detach()
# Initialize SimplEX, fit it on test examples
simplex = Simplex(corpus_examples=corpus_inputs, corpus_latent_reps=corpus_latents)
simplex.fit(
    test_examples=test_inputs,
    n_epoch=50000,
    test_latent_reps=test_latents,
    reg_factor=0,
)

### Compute the SimplEx decomposition for test patient i

In [ ]:
# Compute the Integrated Jacobian for a particular example
i = 1
simplex.jacobian_projection(test_id=i, model=model, input_baseline=input_baseline)
result, sort_order = simplex.decompose(i, return_id=True)

### Display Test Patient
Use the variable `test_time_steps_to_display` to change the number of time steps is the displayed output. A value of 50 or greater will ensure all available time points are displayed.

In [ ]:
# set up dataframes
pd.set_option("display.max_columns", None)
test_time_steps_to_display = 3

# Test patient
test_patient_last_time_step_idx = (
    simplex.test_examples[i][
        ~np.all(simplex.test_examples[i].numpy() == 0, axis=1)
    ].shape[0]
    - 1
)
if (test_patient_last_time_step_idx + 1) - test_time_steps_to_display < 0:
    test_time_steps_to_display = test_patient_last_time_step_idx + 1

test_patient_df = pd.DataFrame(
    simplex.test_examples[i][
        test_patient_last_time_step_idx
        - (test_time_steps_to_display - 1) : test_patient_last_time_step_idx
        + 1,
        :,
    ].numpy(),
    columns=feature_names,
    index=[
        f"(t_max) - {i}" if i != 0 else "(t_max)"
        for i in reversed(range(test_time_steps_to_display))
    ],
)
display(test_patient_df.transpose())

### Display Corpus decomposition and importances

Define styling functions for displaying dataframe.

Define an `example_importance_threshold` to see all the members of the corpus that have an importance above that certain value and
`corpus_time_steps_to_display` to set the number of time steps to include in the output. 

In [ ]:
def df_values_to_colors(df):
    """Gets color values based in values relative to all other values in df."""

    min_val = np.nanmin(df.values)
    max_val = np.nanmax(df.values)
    for col in df:
        # map values to colors in hex via
        # creating a hex Look up table table and apply the normalized data to it
        norm = mcolors.Normalize(
            vmin=min_val,
            vmax=max_val,
            # vmin=np.nanmin(df[col].values),
            # vmax=np.nanmax(df[col].values),
            clip=True,
        )
        lut = plt.cm.bwr(np.linspace(0.2, 0.75, 256))
        lut = np.apply_along_axis(mcolors.to_hex, 1, lut)
        a = (norm(df[col].values) * 255).astype(np.int16)
        df[col] = lut[a]
    return df


def highlight(x):
    return pd.DataFrame(importance_df_colors.values, index=x.index, columns=x.columns)

In [ ]:
# Corpus of patients

# Variables
example_importance_threshold = 0.1
corpus_time_steps_to_display = 10


# Patient Feature values
last_time_step_idx = [
    result[j][1][~np.all(result[j][1].numpy() == 0, axis=1)].shape[0] - 1
    for j in range(len(result))
]


corpus_dfs = [
    pd.DataFrame(
        result[j][1][idx - (corpus_time_steps_to_display - 1) : idx + 1].numpy(),
        columns=feature_names,
    )
    for j, idx in zip(range(len(result)), last_time_step_idx)
]
for corpus_df in corpus_dfs:
    for col_name, rescale_value in rescale_dict.items():
        corpus_df[col_name] = corpus_df[col_name].apply(lambda x: x * rescale_value)
corpus_data = [
    {
        "feature_vals": corpus_dfs[i].transpose(),
        "Label": apply_sort_order(corpus_targets, sort_order)[i],
        "Prediction": apply_sort_order(corpus_predictions, sort_order)[i],
        "Example Importance": result[i][0],
    }
    for i in range(len(corpus_dfs))
]

# Patient importances
importance_dfs = [
    pd.DataFrame(
        result[j][2][idx - (corpus_time_steps_to_display - 1) : idx + 1].numpy(),
        columns=[f"{col}_fi" for col in feature_names],
    )
    for j, idx in zip(range(len(result)), last_time_step_idx)
]
importance_data = [
    {
        "importance_vals": importance_dfs[i].transpose(),
        "Label": apply_sort_order(corpus_targets, sort_order)[i],
        "Prediction": apply_sort_order(corpus_predictions, sort_order)[i],
        "Example Importance": result[i][0],
    }
    for i in range(len(corpus_dfs))
]

corpus_data = [
    example
    for example in corpus_data
    if example["Example Importance"] >= example_importance_threshold
]
importance_data = [
    example
    for example in importance_data
    if example["Example Importance"] >= example_importance_threshold
]

for example_i in range(len(corpus_data)):
    if (last_time_step_idx[example_i] + 1) - corpus_time_steps_to_display < 0:
        corpus_time_steps_to_display = last_time_step_idx[example_i] + 1
    importance_df_colors = df_values_to_colors(
        importance_data[example_i]["importance_vals"].copy()
    )
    importance_df_colors = importance_df_colors.applymap(
        lambda x: f"background-color: {x}"
    )
    display_corpus_df = (
        corpus_data[example_i]["feature_vals"]
        .rename(
            columns={
                j: t_val
                for j, t_val in enumerate(
                    [
                        f"(t_max) - {i}" if i != 0 else "(t_max)"
                        for i in reversed(range(corpus_time_steps_to_display))
                    ]
                )
            }
        )
        .style.apply(highlight, axis=None)
    )
    print(f"Corpus Example: {example_i}")
    print(f"Example Importance: {corpus_data[example_i]['Example Importance']}")
    display(display_corpus_df)

### Save the model to file

In [ ]:
# Save SimplEx model
explainer_path = f"../demonstrator/resources/trained_models/RNN/Time series Prostate Cancer/simplex.pkl"

with open(explainer_path, "wb") as f:
    print(f"Saving SimplEx decomposition in {explainer_path}.")

    pkl.dump(simplex, f)